<a href="https://colab.research.google.com/github/DuarteVn/PySpark-no-Google-Colab/blob/main/Notebook_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Etapa 1: Configurar PySpark e integração com MySQL

In [1]:
from google.colab import drive
drive.mount('/content/drive')
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date, date_format, year, month, concat_ws, sum as _sum

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Baixar o .jar correto do driver JDBC (MySQL Connector 8.0.33)
!wget https://repo1.maven.org/maven2/com/mysql/mysql-connector-j/8.0.33/mysql-connector-j-8.0.33.jar

# Copiar para o diretório do Spark
!cp mysql-connector-j-8.0.33.jar /usr/local/lib/python3.*/dist-packages/pyspark/jars/

--2025-05-23 02:57:42--  https://repo1.maven.org/maven2/com/mysql/mysql-connector-j/8.0.33/mysql-connector-j-8.0.33.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209, 2a04:4e42:4c::209, ...
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2481560 (2.4M) [application/java-archive]
Saving to: ‘mysql-connector-j-8.0.33.jar.2’

mysql-connector-j-8 100%[===================>]   2.37M  --.-KB/s    in 0.05s   

2025-05-23 02:57:42 (46.3 MB/s) - ‘mysql-connector-j-8.0.33.jar.2’ saved [2481560/2481560]



In [3]:
spark = SparkSession.builder.appName("Notebook3_Vendas").getOrCreate()

df = spark.read.csv("/content/drive/MyDrive/Colab Notebooks/Excel/notafiscal.csv", header=True, inferSchema=True, sep=',')
df.printSchema()
df.show(5)

root
 |-- id_venda: integer (nullable = true)
 |-- cod_cliente: integer (nullable = true)
 |-- nom_cliente: string (nullable = true)
 |-- cod_vendedor: integer (nullable = true)
 |-- nom_vendedor: string (nullable = true)
 |-- cod_cidade: integer (nullable = true)
 |-- nom_cidade: string (nullable = true)
 |-- cod_produto: integer (nullable = true)
 |-- Nom_produto: string (nullable = true)
 |-- cod_marca: integer (nullable = true)
 |-- Nom_marca: string (nullable = true)
 |-- cod_departamento: integer (nullable = true)
 |-- nom_departamento: string (nullable = true)
 |-- cod_gerencia: integer (nullable = true)
 |-- nom_gerencia: string (nullable = true)
 |-- dtc_venda: date (nullable = true)
 |-- qtd_venda: integer (nullable = true)
 |-- val_venda: double (nullable = true)
 |-- num_nota: integer (nullable = true)

+--------+-----------+---------------+------------+---------------+----------+-----------+-----------+-----------+---------+-------------+----------------+----------------+-

In [4]:
df_vendas = df.select(
    "id_venda", "cod_cliente", "cod_gerencia","cod_produto", "val_venda", "dtc_venda", "num_nota"
).dropDuplicates()

df_vendas.show(5)

+--------+-----------+------------+-----------+---------+----------+--------+
|id_venda|cod_cliente|cod_gerencia|cod_produto|val_venda| dtc_venda|num_nota|
+--------+-----------+------------+-----------+---------+----------+--------+
|      23|          4|           2|         22|   1600.0|2023-04-12|     750|
|     306|          3|           4|          1|    25.15|2022-10-13|     811|
|     705|          6|           3|         15|     98.5|2023-08-06|     413|
|    1117|          2|           2|         12|      5.3|2021-08-07|     355|
|    1176|          2|           1|         12|      5.3|2022-11-14|    1396|
+--------+-----------+------------+-----------+---------+----------+--------+
only showing top 5 rows



In [5]:
df_vendas.coalesce(1).write.mode("overwrite").option("header", True).csv("/content/vendas_csv")


# Etapa 3: Consolidar soma de vendas pos anos mes cidade e produto

In [6]:
df = df.withColumn("dtc_venda", to_date("dtc_venda"))
df = df.withColumn("Ano-mês", date_format("dtc_venda", "yyyyMM"))

df.show()

+--------+-----------+---------------+------------+---------------+----------+-----------+-----------+----------------+---------+-------------+----------------+----------------+------------+-------------------+----------+---------+---------+--------+-------+
|id_venda|cod_cliente|    nom_cliente|cod_vendedor|   nom_vendedor|cod_cidade| nom_cidade|cod_produto|     Nom_produto|cod_marca|    Nom_marca|cod_departamento|nom_departamento|cod_gerencia|       nom_gerencia| dtc_venda|qtd_venda|val_venda|num_nota|Ano-mês|
+--------+-----------+---------------+------------+---------------+----------+-----------+-----------+----------------+---------+-------------+----------------+----------------+------------+-------------------+----------+---------+---------+--------+-------+
|       1|          6|Reginaldo Rossi|           5|    Maria Braga|         1|   Salvador|          4|         Monitor|        4|          IBM|               1|     Informática|           4|   Casa e Decoração|2021-05-10|  

In [7]:
# Tabela temporaria de teste para verificar quantidade de monitores no ano_mes específico]

df_budapeste_monitor_202310 = df.filter(
    (df["nom_cidade"] == "Budapeste") &
    (df["Nom_produto"] == "Monitor") &
    (df["Ano-mês"] == "202310")
)

df_budapeste_monitor_202310.show()

+--------+-----------+---------------+------------+---------------+----------+----------+-----------+-----------+---------+---------+----------------+----------------+------------+-------------------+----------+---------+---------+--------+-------+
|id_venda|cod_cliente|    nom_cliente|cod_vendedor|   nom_vendedor|cod_cidade|nom_cidade|cod_produto|Nom_produto|cod_marca|Nom_marca|cod_departamento|nom_departamento|cod_gerencia|       nom_gerencia| dtc_venda|qtd_venda|val_venda|num_nota|Ano-mês|
+--------+-----------+---------------+------------+---------------+----------+----------+-----------+-----------+---------+---------+----------------+----------------+------------+-------------------+----------+---------+---------+--------+-------+
|      83|          4|      Tom Jobim|          11| Filipe Almeida|         5| Budapeste|          4|    Monitor|        4|      IBM|               1|     Informática|           4|   Casa e Decoração|2023-10-08|        3|   1300.0|     623| 202310|
|   

In [8]:
df_consilidado_com_qtd = df.groupBy("Ano-mês", "nom_cidade", "Nom_produto") \
    .agg(_sum("qtd_venda").alias("SOMA_QTD_VENDAS"))

df_consilidado_com_qtd.show()

+-------+-----------+--------------------+---------------+
|Ano-mês| nom_cidade|         Nom_produto|SOMA_QTD_VENDAS|
+-------+-----------+--------------------+---------------+
| 202111|Nova Iorque|                Sofá|              4|
| 202211|       Roma|            Papel A4|              5|
| 202210|  Budapeste|Aparalho de Barbe...|              8|
| 202202|   Salvador|             Caneta |              4|
| 202206|   Salvador|            Borracha|              5|
| 202107|   Salvador|          microondas|              7|
| 202309|   Salvador|               Mouse|              5|
| 202209|      Bagdá|           Geladeira|              2|
| 202108|Nova Iorque|           Geladeira|              5|
| 202304|  Budapeste|            Papel A4|              7|
| 202209|   Salvador|         Celular 4Gb|              5|
| 202311|  Budapeste|            Borracha|             10|
| 202204|   Salvador|                Mesa|              9|
| 202210|  Budapeste|           Geladeira|              

In [9]:
df_monitor_budapeste_202310 = df_consilidado_com_qtd.filter(
    (df_consilidado_com_qtd["Ano-mês"] == "202310") &
    (df_consilidado_com_qtd["nom_cidade"] == "Budapeste") &
    (df_consilidado_com_qtd["Nom_produto"] == "Monitor")
)

df_monitor_budapeste_202310.show()

+-------+----------+-----------+---------------+
|Ano-mês|nom_cidade|Nom_produto|SOMA_QTD_VENDAS|
+-------+----------+-----------+---------------+
| 202310| Budapeste|    Monitor|             11|
+-------+----------+-----------+---------------+



In [10]:
df_consilidado_com_qtd.write.mode("overwrite").parquet("/content/consilidado_com_qtd_parquet")
